### 店舗名変換テーブルの作成

In [1]:
import psycopg2
import os
from dotenv import load_dotenv
import pandas as pd
load_dotenv()

users = os.getenv('HEROKU_PSGR_USER')    # DBにアクセスするユーザー名(適宜変更)
dbnames = os.getenv('HEROKU_PSGR_DATABASE')   # 接続するデータベース名(適宜変更)
passwords = os.getenv('HEROKU_PSGR_PASSWORD')  # DBにアクセスするユーザーのパスワード(適宜変更)
host = os.getenv('HEROKU_PSGR_HOST')     # DBが稼働しているホスト名(適宜変更)
port = 5432        # DBが稼働しているポート番号(適宜変更)

# PostgreSQLへ接続
conn = psycopg2.connect("user=" + users +" dbname=" + dbnames +" password=" + passwords, host=host, port=port)

# カーソルを取得する
cursor = conn.cursor()

#自動コミットモードにする
conn.autocommit = True
conn.autocommit

True

In [165]:
cursor.execute('''SELECT 店舗名／法人名, 都道府県,  緯度, 経度
               FROM tenpo_master
               WHERE 経度 is NULL ''')
cols = [col.name for col in cursor.description]
non_latitune_df = pd.DataFrame(cursor.fetchall(),columns=cols)
non_latitune_df

,店舗名／法人名,都道府県,緯度,経度
0,ひまわり北上店,岩手県,None,None
1,グランキコーナ泉佐野店,大阪府,None,None
2,パチンコジャンボ,,None,None
3,APAN CLUB トガリ店,愛知県,None,None
4,メガガイア上越,新潟県,None,None
...,...,...,...,...
208,トキワホール,,None,None
209,パーラーデルダス,,None,None
210,Super D’station前橋大利根店,群馬県,None,None
211,yout小牧店,愛知県,None,None


In [166]:
non_latitune_df

,店舗名／法人名,都道府県,緯度,経度
0,ひまわり北上店,岩手県,None,None
1,グランキコーナ泉佐野店,大阪府,None,None
2,パチンコジャンボ,,None,None
3,APAN CLUB トガリ店,愛知県,None,None
4,メガガイア上越,新潟県,None,None
...,...,...,...,...
208,トキワホール,,None,None
209,パーラーデルダス,,None,None
210,Super D’station前橋大利根店,群馬県,None,None
211,yout小牧店,愛知県,None,None


In [163]:
hallnavi_scraping_error_tenpo_name_dict

{'Fkeiba木更津': IndexError('list index out of range'),
 'プライムナンバーズ株式会社': IndexError('list index out of range'),
 '株式会社日本レジャーチャンネル': IndexError('list index out of range'),
 '株式会社アットワイズ': IndexError('list index out of range'),
 'テスト樋口20190326': IndexError('list index out of range'),
 '株式会社PiDEA': IndexError('list index out of range'),
 'サミー株式会社': IndexError('list index out of range'),
 '株式会社ニューギン販売': IndexError('list index out of range'),
 '株式会社エースプロ': IndexError('list index out of range'),
 'ユニバーサルエンターテインメント': IndexError('list index out of range'),
 'クリーク＆リバー社': IndexError('list index out of range'),
 '株式会社マルハン\u3000東日本カンパニー': IndexError('list index out of range')}

In [ ]:
import pandas as pd
import requests
import time
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
hallnavi_scraping_error_tenpo_name_dict = {}
res = requests.get('https://chromedriver.storage.googleapis.com/LATEST_RELEASE')
options = Options()
options.add_argument('--headless')
browser = webdriver.Chrome(ChromeDriverManager(res.text).install(),options=options)
serched_df_3 = pd.DataFrame(index=[], columns=[])
hallnavi_tenpo_name_dict = {}
serched_df = pd.DataFrame(columns=['dmm_tenpo_name', 'prefecture','hallnavi_tenpo_name', 'url', 'longtitude','latitude'],index=[]) 

i = 0
for i,row in non_latitune_df.iterrows():
    #try:
    tenpo_name = row['店舗名／法人名']
    prefecture = row['都道府県']
    url = f"https://www.google.com/search?q=ホールナビ  {tenpo_name} "
    #print(i,tenpo_name)
    #print('データ取得中...30秒ほど時間がかかります。')
    browser.get(url)
    html = browser.page_source.encode('utf-8')
    soup = BeautifulSoup(html,'html.parser')
    class_group = soup.find_all(class_='yuRUbf')
    #print(len(class_group))
    # リンクとタイトル ,class_group is Noneを抽出しリストに追加するforループ
    for elem in class_group:
        # リンク
        url = elem.a.attrs.get('href')
        # タイトル
        title = elem.a.h3.text.split('｜')[0]
        print(title, url)
        break
    scraping_hallnavi_tenpo_name = title
    browser.get(url)
    
    html = browser.page_source.encode('utf-8')
    url_soup = BeautifulSoup(html,'html.parser')
    try:
        hallnavi_tenpo_name = url_soup.find_all(class_ = 'box_hole_view_hole_name')[0].text
    except Exception as e:
        hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
        print('エラー店舗',tenpo_name,e)
        continue
        
    dfs = pd.read_html(url)
    for df in dfs:
        try:
            check_text = df[0][0]
        except:
            continue
        if check_text == '店舗HP':
            display(df)
            break
    try:
        tenpo_adress = df[df[0] == '住所'][1].values[0]
    except Exception as e:
        hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
        print('エラー店舗',tenpo_name,e)
        continue
    #print(tenpo_adress)

    makeUrl = "https://msearch.gsi.go.jp/address-search/AddressSearch?q="
    s_quote = urllib.parse.quote(tenpo_adress)
    response = requests.get(makeUrl + s_quote)
    #print(response.json()[0]["geometry"]["coordinates"])# [経度,緯度]
    longtitude = response.json()[0]["geometry"]["coordinates"][0]
    latitude = response.json()[0]["geometry"]["coordinates"][1]
    #                  columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude']
    record_df  = pd.DataFrame([[tenpo_name,prefecture,hallnavi_tenpo_name,   url   , longtitude    ,latitude]],columns=['dmm_tenpo_name','prefecture', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'])
    print(record_df)
    serched_df = pd.concat([serched_df,record_df],axis=0,ignore_index=True)

    time.sleep(2)
        
    # except Exception as e:
    #     hallnavi_scraping_error_tenpo_name_dict[tenpo_name] =  e
    #     print('エラー店舗',tenpo_name,e)
    #     break
    #     continue
    # if i > 10:
    #     break
#data_df.to_csv(csv_file_name)

In [164]:
serched_df.to_csv('csv/scrapinged_kanto_tenpo_name_table.csv', index=False)

In [168]:
df

,0,1
0,評価 -0.5,絆の森印西店


In [ ]:
try: 
    try:
        check_text = df[0][0]
    except:
        continue

In [153]:
dfs = pd.read_html(url)
for df in dfs:
    try:
        check_text = df[0][0]
    except:
        continue
    if check_text == '店舗HP':
        display(df)
        break

,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,千葉県千葉市中央区富士見2-4-5
2,電話,043-202-0077
3,パチンコ台数,667
4,パチンコ料金,[4][1]
5,スロット台数,370
6,スロット料金,[1000円/46枚][1000円/184枚]
7,Line,店舗 LINE
8,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


In [142]:
serched_df = pd.DataFrame(columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'],index=[]) 
record_df  = pd.DataFrame([[tenpo_name,hallnavi_tenpo_name,   url   , longtitude    ,latitude]],columns=['dmm_tenpo_name', 'hallnavi_tenpo_name', 'url', 'longtitude','latitude'])
serched_df = pd.concat([serched_df,record_df],axis=0,ignore_index=True)

In [144]:
serched_df

,dmm_tenpo_name,hallnavi_tenpo_name,url,longtitude,latitude
0,ＳＬＯＴ Ｇ８,SLOTG8,https://hall-navi.com/hole_view?hid=2991142000...,139.880066,35.343102


10
SLOTG8 https://hall-navi.com/hole_view?hid=299114200000673112


'SLOTG8'

In [85]:
url_soup.find_all(class_ ='table')

[]

In [92]:
url_soup.text

'\n\n\n\n\n\nSLOTG8｜出玉データや取材・旧イベのまとめとおすすめ情報\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n関東版 \n\n\n\n\n\n\n\n          Menu\n        \n\n\n\n\n\n\n\n\n              プレミアム会員登録\n            \n\n\n\n\n\n\n\n              ログイン\n            \n\n\n\n\n\n\n\n              おすすめ分析\n            \n\n\n\n\n\n\n\n              取材や旧イベを検索\n            \n\n\n\n\n\n\n\n              ホールを検索\n            \n\n\n\n\n\n\n\n              結果レポートを検索\n            \n\n\n\n\n\n\n\n              ユーザー投稿一覧\n            \n\n\n\n\n\n\n\n              エリア選択\n            \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n7/29(土)おすすめランキング\n\n\n東京都\n17.5点\n\n\n\n\n\n\n\n\n神奈川県\n19.2点\n\n\n\n\n\n\n\n\n埼玉県\n18.6点\n\n\n\n\n\n\n\n\n千葉県\n19.7点\n\n\n\n\n\n\n\n\n茨城県\n17.8点\n\n\n\n\n\n\n\n\n栃木県\n16.3点\n\n\n\n\n\n\n\n\n群馬県\n13.8点\n\n\n\n\n\n7/29(土)スケジュールエリアから探す\n\n\n\n                                東京都\n                            \n                                1

In [124]:

serch_df  

,dmm_tenpo_name,hallnavi_tenpo_name,url,longtitude,latitude


In [102]:
df[0][0] == '店舗HP'

False

,0,1
0,店舗HP,DMMぱちタウン / P-WORLD
1,住所,千葉県君津市坂田673-11
2,スロット台数,224
3,スロット料金,[20]
4,その他,設置機種・営業状況はPWORLD等をご確認下さい。店休日には対応していません 店舗名でGoo...


千葉県君津市坂田673-11
[139.880066, 35.343102]


'千葉県君津市坂田673-11'

In [31]:


#リンク取得「a」タグ
for res in soup.find_all('a'):
    print(res.get('href'))
    break

/?sa=X&ved=0ahUKEwi_uNSPx7GAAxUZGjQIHc_2AN4QOwgC


In [16]:
import re
import requests

from bs4 import BeautifulSoup
googleSearch = 'https://www.google.co.jp/search'
response = requests.get(googleSearch, params={'q': f'ホールナビ  {tenpo_name} ','num':5})
html = response.text.encode()
bs4 = BeautifulSoup(html,'html.parser')
searchResults = bs4.select('.r > a')
for searchResult in searchResults:
    title = searchResult.text
    url = re.sub("\/url\?q=","",searchResult.get('href'))
    print('タイトル：',title)
    print('URL：',url)

In [18]:
searchResults

[]

In [ ]:

cursor.execute('''SELECT *
               FROM maptable
               WHERE prefecture = '神奈川県' AND latitude is NULL ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)





In [36]:
cursor = conn.cursor()
cursor.execute('''SELECT COUNT(DISTINCT 店舗名)
               FROM parlar_data
               WHERE 都道府県 = '神奈川県' or '埼玉県' ''')
cols = [col.name for col in cursor.description]
maptable_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(maptable_df)

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [ ]:
cursor.execute('''SELECT * 
               FROM schedule 
               ORDER BY 取得時間 DESC
               LIMIT 10''')
cols = [col.name for col in cursor.description]
schedule_df = pd.DataFrame(cursor.fetchall(),columns=cols)
display(schedule_df)